In [2]:
import pandas as pd
pd.read_csv('data/globalterrorismdb_0617dist.csv', header=0, encoding='ISO-8859-1').shape


/Users/Paul/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(170350, 135)

# Machine Learning for Counterterrorism
**Authors:** Akhil Jalan, Yakub Akhmerov, Paul Kim

# Abstract

Terrorism, defined broadly as "the use of intentionally indiscriminate violence as a means to create terror, or fear, to achieve a political, religious or ideological aim", is on the rise. In order to assist the fight against terrorism, the University of Maryland's Department of Homeland Security Center of Excellence compiled an extensive and rich database of international terrorist attacks from 1970 to 2016, to be used for analysis and data science purposes. 

In order to assist the fight against terror, we sought to answer a critical question: what factors are most influential in determining whether a terrorist attack succeeds or fails. Armed with this information, counterterrorist organizations would be better able to prioritize their foci for further research and training. 

# Methods

## Preprocessing

The raw data came in the form of two large csv files, with a total of 170350 entries spanning 135 variables with information on the date, attack type, location, tactics, perpetrating groups, targets, and outcomes. 

In order to preprocess the data, we made a python file with convenience preprocessing functions that could be run at the start of each analysis notebook. These preprocessing functions loaded raw csv data into pandas, dropped unused columns or columns with a large number of NAs, and created one-hot-encoded dummy variables out of the important categorical features. 

We could have made a separate notebook that, when executed, outputted the cleaned data and saved it to file, which could then be loaded in subsequent notebooks. However, because the cleaning process for this is very quick, and it requires less lines of code to execute the preprocessing functions, we decided to simply run the preprocessing functions in every notebook. In addition, the models in the analysis uses slightly different subsets of the data, and as such it makes more sense to preprocess the data separately for each model. 

## Analysis:

In order to 